<a href="https://colab.research.google.com/github/teamgaon/recommendation_algorithms_know/blob/main/220118_sm_know_submission3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 전공을 임베딩

train set으로 성능 확인용 

In [1]:
!pip install gluonnlp
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip install transformers
!pip install sentencepiece
!pip install kobert-transformers
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-tsq2yeys/kobert-tokenizer_c192b268d45d4dbbb958c657639e20f6
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-tsq2yeys/kobert-tokenizer_c192b268d45d4dbbb958c657639e20f6
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-_2s1yj07
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-_2s1yj07


In [2]:
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
from google.colab import drive
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

import pandas as pd
from glob import glob
from tqdm import tqdm
import warnings

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from kobert_tokenizer import KoBERTTokenizer

In [3]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
warnings.filterwarnings(action='ignore')

In [32]:
train1 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2017.csv', encoding='utf-8')
train2 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2018.csv', encoding='utf-8')
train3 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2019.csv', encoding='utf-8')
train4 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2020.csv', encoding='utf-8')

train2 = train2[train2['idx'] != 18048]
train3 = train3[train3['idx'] != 19871]
train3 = train3[train3['idx'] != 25823]
train3 = train3[train3['idx'] != 18746]
train3 = train3[train3['idx'] != 19183]
train3 = train3[train3['idx'] != 21879]
train3 = train3[train3['idx'] != 21931]
train3 = train3[train3['idx'] != 26330]

test1 = train1[int(len(train1)*0.8):]
train1 = train1[:int(len(train1)*0.8)]
test2 = train2[int(len(train2)*0.8):]
train2 = train2[:int(len(train2)*0.8)]
test3 = train3[int(len(train3)*0.8):]
train3 = train3[:int(len(train3)*0.8)]
test4 = train4[int(len(train4)*0.8):]
train4 = train4[:int(len(train4)*0.8)]

test2 = test2[test2['idx'] != 17700]

In [6]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

Downloading:   0%|          | 0.00/363k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/244 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/432 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [33]:
def text_to_token(text:str):
  temp = tokenizer.encode(text)

  return temp[1]

In [34]:
train3['bq27_1'].replace(' ', '없음', inplace=True)

In [35]:
train3['bq27_1'] = train3['bq27_1'].fillna('없음')

In [36]:
train1['bq38_1'] = train1['bq38_1'].map(text_to_token)
train2['bq37_1'] = train2['bq37_1'].map(text_to_token)
train3['bq27_1'] = train3['bq27_1'].map(text_to_token)
train4['bq26_1'] = train4['bq26_1'].map(text_to_token)

In [37]:
test1['bq38_1'] = test1['bq38_1'].map(text_to_token)
test2['bq37_1'] = test2['bq37_1'].map(text_to_token)
test3['bq27_1'] = test3['bq27_1'].map(text_to_token)
test4['bq26_1'] = test4['bq26_1'].map(text_to_token)

In [38]:
know_train = [train1, train2, train3, train4]
know_test = [test1, test2, test3, test4]

In [39]:
for df in know_train:
    for col in df.columns:
        df[col].replace(' ', '0', inplace=True)

In [40]:
for df in know_test:
    for col in df.columns:
        df[col].replace(' ', '0', inplace=True)

In [41]:
years = ['2017', '2018', '2019', '2020']

rf_predicts = []
year_encoder = {}
int_col = {}
text_col = {}

for year, train, test in zip(years, know_train, know_test):
    print(year)
    encoders = {}
    int_temp = []
    text_temp = []
    for col in train.columns:
        if col == 'ID':
            continue
        try:
            train[col] = train[col].map(int)
            int_temp.append(col)
        except:
            encoder = LabelEncoder()
            train[col] = train[col].map(str)
            train[col] = encoder.fit_transform(train[col])
            encoders[col] = encoder
            text_temp.append(col)
    year_encoder[year] = encoders
    int_col[year] = int_temp
    text_col[year] = text_temp

    feature = train.columns.drop(['idx', 'knowcode'])

    model = RandomForestClassifier(n_estimators=100, random_state=123456, n_jobs=8)
    model.fit(train[feature], train['knowcode'])

    for col in test.columns:
      try:
          test[col] = test[col].map(int)
      except:
          encoder = year_encoder[year][col]
          test[col] = test[col].map(str)
          category_map = {category: idx for idx, category in enumerate(encoder.classes_)}
          test[col] = test[col].apply(lambda x: category_map[x] if x in category_map else -1) # train set에서 보지못한 카테고리변수 -1(UNK) 처리
    feature = test.columns.drop(['idx', 'knowcode'])
    pred = model.predict(test[feature])
    rf_predicts.extend(pred)

2017
2018
2019
2020


In [46]:
len(rf_predicts)

7047

In [47]:
df = pd.concat([test1[['idx', 'knowcode']], test2[['idx', 'knowcode']], test3[['idx', 'knowcode']], test4[['idx', 'knowcode']]], axis=0)

In [49]:
df['predict'] = rf_predicts

In [52]:
len(df[df['knowcode'] == df['predict']])/len(df)

0.4616148715765574